<a href="https://colab.research.google.com/github/mateuszrajca69/Chess/blob/master/kontenery_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
pip install --user ortools

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Poniższy kod importuje wymagane biblioteki.
from ortools.linear_solver import pywraplp

In [ ]:
def create_data_model():
    """tworzymy dane dla przykładu"""
    data = {}
    weights = [48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30,48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30,48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30,2,3,3,3,1,55,99,89,12,6,10,11,23,33,56,78,90,10,8,7,7,7,6,6,13,17,22,32,24,25,5,7,8,9,1,10,48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30,48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30,48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30,2,3,3,3,1,55,99,89,12,6,10,11,23,33,56,78,90,10,8,7,7,7,6,6,13,17,22,32,24,25,5,7,8,9,1,10]    
    #wagi: wektor zawierający wagi elementów.
    #dla 138 przedmiotów 
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['bins'] = data['items']                              
    data['bin_capacity'] = 100                                #bin_capacity: pojedyncza liczba określająca pojemność pojemników.
    return data


    #maksymalna liczba pojemników, których można potrzebować, to liczba przedmiotów, ponieważ zawsze można umieścić każdy przedmiot w oddzielnym koszu.



    
def main():
    data = create_data_model()


# zastosowujemy solver do programowania mieszanych liczb całkowitych. Poniższy kod deklaruje solver.
    solver = pywraplp.Solver.CreateSolver('SCIP')


   
# x [i, j] = 1, jeśli element i jest zapakowany do pojemnika j.
    x = {}
    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

   
# y [j] = 1, jeśli używany jest bin j.
    y = {}
    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

#Definiuje tablicę zmiennych x [(i, j)] których wartość wynosi 1, jeśli element i zostanie umieszczony w bin j, a 0 w przeciwnym razie.
#W przypadku pakowania w kontener definiuje się również tablicę zmiennych y [j] której wartość wynosi 1, jeśli używany jest kontener(przedział) j - to znaczy 
#jeśli są w niej spakowane jakiekolwiek elementy - i 0 w przeciwnym razie. Suma y [j] będzie liczbą użytych przedziałów (konetnerów).

 
  # definiujemy ograniczenia problemu:
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)

# Ilość umieszczona w każdym pojemniku nie może przekraczać jego pojemności.
    
    
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
            data['bin_capacity'])
#Każdy przedmiot należy umieścić w dokładnie jednym pojemniku. To ograniczenie jest ustalane przez wymaganie, aby suma x [i] [j] na wszystkich pojemnikach j była równa 1. 

   
# Cel: zminimalizować liczbę używanych pojemników.
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))



#Poniższy kod wywołuje solver i wyświetla rozwiązanie.
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0.
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data['weights'][i]
                if bin_weight > 0:
                    num_bins += 1
                    print('numer pojemnika/kontenera :', j)
                    print('  spakowane przedmioty:', bin_items)
                    print('  ostateczna waga przedmiotów:', bin_weight)
                    print()
        print()
        print('Ilość użytych pojemników:', num_bins)
        print('Time = ', solver.WallTime(), ' milliseconds')
    else:
        print('Problem nie ma optymalnego rozwiązania.')


if __name__ == '__main__':
    main()

numer pojemnika/kontenera : 0
  spakowane przedmioty: [0, 1, 2, 33, 37]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 1
  spakowane przedmioty: [3, 4, 5, 67]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 2
  spakowane przedmioty: [6, 7, 34, 35, 36, 42, 106]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 3
  spakowane przedmioty: [8, 9, 10, 43]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 4
  spakowane przedmioty: [11, 12, 13, 102, 136]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 5
  spakowane przedmioty: [14, 15, 16]
  ostateczna waga przedmiotów: 99

numer pojemnika/kontenera : 6
  spakowane przedmioty: [17, 18, 41, 103]
  ostateczna waga przedmiotów: 99

numer pojemnika/kontenera : 7
  spakowane przedmioty: [19, 20, 21, 50]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 8
  spakowane przedmioty: [22, 23, 24, 104]
  ostateczna waga przedmiotów: 100

numer pojemnika/kontenera : 

In [ ]:
#Rozwiązanie pokazuje minimalną liczbę pojemników potrzebną do zapakowania wszystkich artykułów. 
#Dla każdego używanego pojemnika rozwiązanie pokazuje zapakowane w nim elementy i całkowitą wagę pojemnika.

In [ ]:
#1 mili sekunda to 1/1000 sekundy
#15ms = 0,02s dla 18 tez 

In [ ]:
# program do obliczania ms : https://www.convertworld.com/pl/czas/milisekundy.html